In [20]:
from experiments.mlp import MLPExperiment
import torch
import os

In [18]:
os.chdir('c:\\Users\\Shiyan Liu\\GitHub\\biological-deep-learning')

In [21]:
experiment=MLPExperiment(None, 784, 256 , 10, lamb=1, num_epochs=1, K=1000, heb_lr=0.1)
experiment.train()

FileNotFoundError: [Errno 2] No such file or directory: '../data/mnist/mnist_train.csv'